1. Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import sklearn
from sklearn.linear_model import Ridge, Lasso
from sklearn.neural_network import MLPRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings('ignore')

2. Define functions to load data, create features, create target, and scoring function.

In [3]:
def create_features(df):
    df['Spread'] = df['High'] - df['Low']
    df['Gap'] = df['Open'] - df['Close'].shift(1)
    df['Intraday'] = df['Open'] - df['Close']
    return df

def drop_features(df):
    df.drop(columns=['Spread',
                     'Gap',
                     'Intraday',
                     ],
            inplace=True)

    df.drop(columns=['Open','High','Low','Close','Volume','Adj Close',
                     ], inplace=True)
    return df

def process_features(df, lookback, step):
    for i in range(step, lookback+1, step):
        df['%d Spread' % (i)] = df['Spread'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Spread' % (i)] = df['Spread'].rolling(window=i).mean()

        df['%d Gap' % (i)] = df['Gap'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Gap' % (i)] = df['Gap'].rolling(window=i).mean()

        df['%d Intraday' % (i)] = df['Intraday'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Intraday' % (i)] = df['Intraday'].rolling(window=i).mean()
    return df

def features(df, lookback, step):
    create_features(df)
    process_features(df, lookback, step)
    drop_features(df)
    return df

def create_target(df, lookforward=1, target='Close'):
    df['Target'] = df[target].pct_change(lookforward).shift(-lookforward)
    return df

def custom_score(y_true, y_pred):
  pred_sign = np.sign(y_pred)
  y_true = np.squeeze(y_true)
  returns = np.where((pred_sign == 1), y_true, 0)
  returns = np.where((pred_sign == -1), y_true * -1, returns)
  return returns.mean()
custom_scorer = make_scorer(custom_score, greater_is_better=True)

3. Define the models we are going to use

In [4]:
estimator1 = Ridge()
estimator2 = Lasso(alpha=.001)
estimator3 = LGBMRegressor(random_state=0)
estimator4 = MLPRegressor(random_state=0, solver='adam', activation='relu', shuffle=False)
models = [estimator1,estimator2,estimator3,estimator4]
estimator = VotingRegressor(estimators=[('Ridge', estimator1),
                                        ('Lasso', estimator2),
                                        ('LGBM', estimator3),
                                        ('MLP', estimator4),
                                        ])

4. Define target, cross validation folds, interval, and lookback parameters.

In [5]:
lookforward = 1
tscv = TimeSeriesSplit(n_splits=5, gap=lookforward)
step = 2
lookback = 6

5. Load data

In [6]:
spy = yf.download('SPY', start='2004-12-01')
agg = yf.download('AGG', start='2004-12-01')
gld = yf.download('GLD', start='2004-12-01')

spy = create_target(spy, lookforward, target='Close')

spy = features(spy, lookback, step)
spy = spy.add_suffix(' SPY')
agg = features(agg, lookback, step)
agg = agg.add_suffix(' AGG')
gld = features(gld, lookback, step)
gld = gld.add_suffix(' GLD')
cv = pd.concat([spy,agg,gld], axis=1)

cv.drop(cv.tail(lookforward).index, inplace=True)
cv.drop(cv.head(lookback).index, inplace=True)
X = cv
y = X[['Target SPY']]
X = X.drop(columns=['Target SPY'])
X.fillna(method="ffill", inplace=True)
X.replace([np.inf, -np.inf], 0, inplace=True)
X.fillna(0, inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


6. Define parameter grid. (Look at models on sklearns docs to find which parameters of a model you can change)

In [14]:
param_grid = {
    'Ridge__alpha': [1,.5,.25,.125,0.0625,0.03125,0.015625,0.0078125,0.00390625,0.001953125,0.0009765625,0.00048828125],
    'Lasso__alpha': [1,.5,.25,.125,0.0625,0.03125,0.015625,0.0078125,0.00390625,0.001953125,0.0009765625,0.00048828125],
    'LGBM__max_depth': [1,2,3,4,5,6],
    'LGBM__num_leaves': [2,4,8,16,32,64],
    'LGBM__linear_tree': [True],
    'LGBM__num_iterations': [1,2,4,8,16,32,64,128],
    'LGBM__learning_rate': [1,.5,.25,.125,0.0625,0.03125,0.015625,0.0078125,0.00390625,0.001953125,0.0009765625,0.00048828125],
    'MLP__hidden_layer_sizes': [(1,),(1,1),(1,1,1,1),(2,),(2,2),(2,2,2,2),(4,),(4,4),(4,4,4,4),(8,),(8,8),(8,8,8,8),(16,),(16,16),(16,16,16,16)],
}

7. Split train and test data and run Random Search on train data

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, shuffle=False)
search = RandomizedSearchCV(estimator, param_distributions=param_grid, n_iter=400, cv=tscv, scoring=custom_scorer, n_jobs=-1, verbose=1)
search.fit(X_train, y_train)

Fitting 5 folds for each of 400 candidates, totalling 2000 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=1, max_train_size=None, n_splits=5, test_size=None),
                   estimator=VotingRegressor(estimators=[('Ridge',
                                                          Ridge(alpha=0.25)),
                                                         ('Lasso',
                                                          Lasso(alpha=0.0078125)),
                                                         ('LGBM',
                                                          LGBMRegressor(learning_rate=0.5,
                                                                        linear_tree=True,
                                                                        max_depth=3,
                                                                        num_iterations=4,
                                                                        num_leaves=64,
                                                                        random_state=0)),
                                                         ('MLP',
                                                          MLPRegressor(hidden_layer_sizes=(2,)...
                                        'LGBM__num_leaves': [2, 4, 8, 16, 32,
                                                             64],
                                        'Lasso__alpha': [1, 0.5, 0.25, 0.125,
                                                         0.0625, 0.03125,
                                                         0.015625, 0.0078125,
                                                         0.00390625,
                                                         0.001953125,
                                                         0.0009765625,
                                                         0.00048828125],
                                        'MLP__hidden_layer_sizes': [(2,)],
                                        'Ridge__alpha': [1, 0.5, 0.25, 0.125,
                                                         0.0625, 0.03125,
                                                         0.015625, 0.0078125,
                                                         0.00390625,
                                                         0.001953125,
                                                         0.0009765625,
                                                         0.00048828125]},
                   scoring=make_scorer(custom_score), verbose=1)

8. Print best parameters

In [16]:
def report(results, n_top=5):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.6f} (std: {1:.6f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")
report(search.cv_results_)
#0.000659 (std: 0.000376)

Model with rank: 1
Mean validation score: 0.000702 (std: 0.000403)
Parameters: {'Ridge__alpha': 0.0009765625, 'MLP__hidden_layer_sizes': (2,), 'Lasso__alpha': 0.25, 'LGBM__num_leaves': 64, 'LGBM__num_iterations': 16, 'LGBM__max_depth': 4, 'LGBM__linear_tree': True, 'LGBM__learning_rate': 0.5}

Model with rank: 2
Mean validation score: 0.000687 (std: 0.000406)
Parameters: {'Ridge__alpha': 0.00390625, 'MLP__hidden_layer_sizes': (2,), 'Lasso__alpha': 0.00390625, 'LGBM__num_leaves': 4, 'LGBM__num_iterations': 2, 'LGBM__max_depth': 6, 'LGBM__linear_tree': True, 'LGBM__learning_rate': 1}

Model with rank: 3
Mean validation score: 0.000686 (std: 0.000420)
Parameters: {'Ridge__alpha': 0.0078125, 'MLP__hidden_layer_sizes': (2,), 'Lasso__alpha': 0.00048828125, 'LGBM__num_leaves': 2, 'LGBM__num_iterations': 4, 'LGBM__max_depth': 4, 'LGBM__linear_tree': True, 'LGBM__learning_rate': 1}

Model with rank: 4
Mean validation score: 0.000667 (std: 0.000394)
Parameters: {'Ridge__alpha': 0.125, 'MLP__hidd

9. Backtest parameters

In [9]:
# !pip3 install backtesting
from backtesting import Strategy, Backtest
from sklearn.model_selection import train_test_split

Loading BokehJS ...

In [17]:
step = 2
lookback = 6

# fit best parameters
estimator1 = Ridge(alpha=0.0009765625)
estimator2 = Lasso(alpha=0.25)
estimator3 = LGBMRegressor(max_depth=4, num_leaves=64, random_state=0, linear_tree=True, num_iterations=16, learning_rate=0.5)
estimator4 = MLPRegressor(hidden_layer_sizes=(2,), random_state=0, solver='adam', activation='relu', shuffle=False)
models = [estimator1,
          estimator2,
          estimator3,
          estimator4,
          ]
estimator = VotingRegressor(estimators=[('Ridge', estimator1),
                                        ('Lasso', estimator2),
                                        ('LGBM', estimator3),
                                        ('MLP', estimator4),
                                        ],)

X_test = X_test.iloc[(abs(lookforward)):]
y_test = y_test.iloc[(abs(lookforward)):]

estimator.fit(X_train, y_train)
forecasted = estimator.predict(X_test)
# # flip sign of forecasted values
# forecasted = forecasted * -1

data = yf.download('SPY', start='2004-12-01')
data.drop(data.tail(lookforward).index,inplace=True)
data.drop(data.head(lookback).index,inplace=True)
data = data.iloc[(-X_test.shape[0]):]
data['forecastedValue'] = forecasted
prediction = data

class MyStrategy(Strategy):
    Data = prediction

    def init(self):
        super().init()

    def next(self):
        if self.data.forecastedValue < 0:
            self.sell()
        elif self.data.forecastedValue > 0:
            self.buy()


bt = Backtest(prediction, MyStrategy,
              cash=1000,
              trade_on_close=True,
              exclusive_orders=True
              )
print(bt.run())
bt.plot()

[*********************100%***********************]  1 of 1 completed
Start                     2021-08-03 00:00...
End                       2023-06-07 00:00...
Duration                    673 days 00:00:00
Exposure Time [%]                   99.569892
Equity Final [$]                  1440.009918
Equity Peak [$]                   1447.239929
Return [%]                          44.000992
Buy & Hold Return [%]               -3.309533
Return (Ann.) [%]                   21.849535
Volatility (Ann.) [%]               20.376378
Sharpe Ratio                         1.072297
Sortino Ratio                        1.988997
Calmar Ratio                         2.002306
Max. Drawdown [%]                  -10.912183
Avg. Drawdown [%]                    -2.11774
Max. Drawdown Duration      114 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                  463
Win Rate [%]                         53.99568
Best Trade [%]                       5.495415
Worst Trade

Row(id='2098', ...)

The annualized return is a little bit worse than the previous notebook, but the strategy beats buying and holding. Would I run this strategy? No, but this is a good starting point. Another interesting thing to point out is that the neural network only has 1 hidden layer with 2 neurons, which is almost one of the simplest possible neural network (except for having one layer with 1 neuron). This is a good example of how more complexity does not always mean better results.

Somethings to explore futher: Create more features, add more data sources, evaluate more models, evaluate more parameters, evaluate higher period step interval and look back periods, evaluate how many cross validation folds are optimal when taking the bias-variance trade-off into account, backtest with commission, etc.